<a href="https://colab.research.google.com/github/ML4SCI/ML4SCIHackathon/blob/main/ParticleImagesChallenge/ParticleImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hackathon - Particle Images
## Problem Statement 
*   One of the important aspects of searches for new physics at the Large Hadron Collider (LHC) involves the classification of various High-Energy Particles in collision events
*   The goal of this challenge is to develop a model which classifies electron and photon electromagnetic showers as accurately as possible based on the detector images provided in the dataset below (one pixel = one channel of the detector)
*   The preferred metric for evaluating the model is ROC curve (Receiver Operating Characteristic curve) and the AUC (Area Under the ROC Curve) score.
*   Although we are using Keras Framework in this sample notebook, you are free to choose Machine Learning / Deep Learning Framework of your choice. 



## Create the appropriate project folder 

In [ ]:
!mkdir Particle_Images

In [ ]:
!cd Particle_Images

In [ ]:
!mkdir data/

# Download the Dataset

In [ ]:
#!/bin/bash
!wget https://cernbox.cern.ch/index.php/s/sHjzCNFTFxutYCj/download -O data/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5
!wget https://cernbox.cern.ch/index.php/s/69nGEZjOy3xGxBq/download -O data/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5

# Import modules

In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility
import h5py
from keras.layers import MaxPool2D, Input, Dense, Dropout, Flatten, Conv2D, AveragePooling2D, concatenate
from keras.models import Model,load_model
from keras.initializers import Constant
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
from keras.optimizers import Adam
from keras.metrics import AUC
from keras.utils import to_categorical
import pickle

from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt

# Keras Model Parameters

In [ ]:
lr_init     = 1.e-3    # Initial learning rate  
batch_size  = 500       # Training batch size
train_size  = 100000     # Training size
valid_size  = 50000     # Validation size
test_size   = 5000     # Test size
epochs      = 100       # Number of epochs
doGPU       = False    # Use GPU

## It is recommended to use GPU for training and inference if possible.

In [ ]:
if doGPU:
    import tensorflow.compat.v1 as tf
    from tensorflow.compat.v1.keras.backend import set_session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    set_session(tf.Session(config=config))

# Load Image Data
### Two classes of particles: electrons and photons
### 32x32 matrices (two channels - hit energy and time) for the two classes of particles electrons and photons impinging on a calorimeter (one calorimetric cell = one pixel).
#### Please note that although timing channel is provided, it may not necessarily help the performance of the model.

In [ ]:
img_rows, img_cols, nb_channels = 32, 32, 2        
input_dir = 'data'
decays = ['SinglePhotonPt50_IMGCROPS_n249k_RHv1', 'SingleElectronPt50_IMGCROPS_n249k_RHv1']

def load_data(decays, start, stop):
    dsets = [h5py.File('%s/%s.hdf5'%(input_dir,decay)) for decay in decays]
    X = np.concatenate([dset['/X'][start:stop] for dset in dsets])
    y = np.concatenate([dset['/y'][start:stop] for dset in dsets])
    assert len(X) == len(y)
    return X, y

# Configure Training / Validation / Test Sets

In [ ]:
# Set range of training set
train_start, train_stop = 0, train_size
assert train_stop > train_start
assert (len(decays)*train_size) % batch_size == 0
X_train, y_train = load_data(decays,train_start,train_stop)

# Set range of validation set
valid_start, valid_stop = 160000, 160000+valid_size
assert valid_stop  >  valid_start
assert valid_start >= train_stop
X_valid, y_valid = load_data(decays,valid_start,valid_stop)

# Set range of test set
test_start, test_stop = 204800, 204800+test_size
assert test_stop  >  test_start
assert test_start >= valid_stop
X_test, y_test = load_data(decays,test_start,test_stop)

samples_requested = len(decays) * (train_size + valid_size + test_size)
samples_available = len(y_train) + len(y_valid) + len(y_test)
assert samples_requested == samples_available

### We only use the energy channel, and we vectorize the target labels to be able to use a softmax output activation

In [ ]:
X_train = X_train[:,:,:,0]
y_train = to_categorical(y_train)

X_valid = X_train[:,:,:,0]
y_valid = to_categorical(y_valid)

X_test = X_test[:,:,:,0]
y_test = to_categorical(y_test)

# Define BearNet Model
## Based on GoogleNet

In [ ]:

kernel_init = 'TruncatedNormal'
bias_init = Constant(value=5e-4)


def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):

    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu',
                      kernel_initializer=kernel_init, bias_initializer=bias_init)(x)

    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu',
                      kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu',
                      kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu',
                      kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu',
                      kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu',
                       kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5,
                         pool_proj], axis=3, name=name)

    return output


def bearnet_truncated(input):
    x = Conv2D(32, (3, 3), padding='same', strides=1,
               activation='relu', name='conv_1_3x3')(input)
    x = MaxPool2D((2, 2), padding='same', strides=2, name='max_pool_1_3x3')(x)
    x = Conv2D(32, (2, 2), padding='same', strides=(1, 1),
               activation='relu', name='conv_2a_2x2')(x)
    x = Conv2D(96, (2, 2), padding='same', strides=(1, 1),
               activation='relu', name='conv_2b_2x2')(x)
    x = MaxPool2D((2, 2), padding='same', strides=1,
                  name='max_pool_2_2x2/2')(x)

    x = inception_module(x,
                         filters_1x1=32,
                         filters_3x3_reduce=48,
                         filters_3x3=64,
                         filters_5x5_reduce=16,
                         filters_5x5=16,
                         filters_pool_proj=16,
                         name='inception_3a')
    x = inception_module(x,
                         filters_1x1=64,
                         filters_3x3_reduce=64,
                         filters_3x3=96,
                         filters_5x5_reduce=16,
                         filters_5x5=48,
                         filters_pool_proj=32,
                         name='inception_3b')

    x = MaxPool2D((2, 2), padding='same', strides=(
        2, 2), name='max_pool_3_2x2/2')(x)
    x = inception_module(x,
                         filters_1x1=96,
                         filters_3x3_reduce=48,
                         filters_3x3=104,
                         filters_5x5_reduce=16,
                         filters_5x5=24,
                         filters_pool_proj=32,
                         name='inception_4a')

    x1 = AveragePooling2D((2, 2), strides=2)(x)
    x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
    x1 = Flatten()(x1)
    x1 = Dense(1024, activation='relu')(x1)
    x1 = Dropout(0.4)(x1)
    x1 = Dense(2, activation='softmax', name='output')(x1)
    model = Model(input, [x1], name='inception_v1')
    return model


input = Input(shape=(32, 32, 1))
model = bearnet_truncated(input)

model.summary()


## Train the Model

In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=2, min_lr=1.e-6)
earlystop = EarlyStopping(monitor='loss', patience=3)

model.compile(loss='binary_crossentropy', optimizer=Adam(
    learning_rate=lr_init), metrics=['accuracy', AUC()])

to_fit = False

if to_fit:
    history = model.fit(
        X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_valid, y_valid),
        callbacks=[reduce_lr, earlystop],
        verbose=1, shuffle=True
    )


## Load the Trained Model

In [ ]:
modeldir = 'bearnet-truncated'
model = load_model(modeldir)
with open(f'{modeldir}/history.pkl','rb') as f_history: history = pickle.load(f_history)

model.summary()

## Evaluate the Model  

In [ ]:
# Evaluate on validation set
score = model.evaluate(X_valid, y_valid, verbose=1)
print('\nValidation loss / accuracy: %0.4f / %0.4f'%(score[0], score[1]))
y_pred = model.predict(X_valid)
fpr, tpr, _ = roc_curve(y_valid, y_pred)
roc_auc = auc(fpr, tpr)
print('Validation ROC AUC:', roc_auc)

# Evaluate on test set
score = model.evaluate(X_test, y_test, verbose=1)
print('\nTest loss / accuracy: %0.4f / %0.4f'%(score[0], score[1]))
y_pred = model.predict(X_test)
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
print('Test ROC AUC:', roc_auc)

In [ ]:
plt.plot([0, 1], [0, 1], 'k--')
#plt.legend(loc=2, prop={'size': 15})
plt.plot(fpr, tpr, label='Model 1 (ROC-AUC = {:.3f})'.format(roc_auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

# Submission format: 
### Please submit the Google Colab Jupyter Notebook demonstrating your solution in the similar format as illustrated in this notebook. It should contain :
*   The final model architecture, parameters and hyper-parameters yielding the best possible performance,
*   Its Training and Validation accuracy, 
*   ROC curve and the AUC score as shown above.
*   Also, please submit the final trained model containing the model architecture and its trained weights along with this notebook (For example: HDF5 file, .pb file, .pt file, etc.). You are free to choose Machine Learning Framework of your choice. 